In [ ]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.io import VTKFile
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot

L = 20.0
domain = mesh.create_box(
    MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron
)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim,)))

[2026-02-27 11:21:34.522] [info] Extract basic topology: 4000->4000
[2026-02-27 11:21:34.522] [info] Build local dual graphs, re-order cells, and compute process boundary vertices.
[2026-02-27 11:21:34.522] [info] Build local part of mesh dual graph (mixed)
[2026-02-27 11:21:34.522] [info] GPS pseudo-diameter:(28) 499-0
[2026-02-27 11:21:34.523] [info] Create topology (generalised)
[2026-02-27 11:21:34.523] [info] Computing communication graph edges (using NBX algorithm). Number of input edges: 1
[2026-02-27 11:21:34.523] [info] Finished graph edge discovery using NBX algorithm. Number of discovered edges 1
[2026-02-27 11:21:34.523] [info] Compute ghost indices
[2026-02-27 11:21:34.523] [info] Computing communication graph edges (using PCX algorithm). Number of input edges: 0
[2026-02-27 11:21:34.523] [info] Finished graph edge discovery using PCX algorithm. Number of discovered edges 0
[2026-02-27 11:21:34.523] [info] Computing communication graph edges (using NBX algorithm). Number o

In [3]:
def left(x):
    return np.isclose(x[0], 0)

    
def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

In [4]:
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(
    domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets]
)

In [5]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

In [6]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
right_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(2))
bcs = [fem.dirichletbc(u_bc, left_dofs, V), fem.dirichletbc(u_bc, right_dofs, V)]

In [7]:
rho = 1.0
g = 9.81
B = fem.Constant(domain, default_scalar_type((0, 0, -rho * g)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

In [8]:
v = ufl.TestFunction(V)
u = fem.Function(V)

In [9]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

In [10]:
E = default_scalar_type(1.0e3)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))

In [11]:
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J)) ** 2

In [12]:
P = ufl.diff(psi, F)

In [13]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

In [14]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)

In [15]:
residual = (
    ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)
)

In [16]:
petsc_options = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "none",
    "snes_monitor": None,
    "snes_atol": 1e-8,
    "snes_rtol": 1e-8,
    "snes_stol": 1e-8,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}
problem = NonlinearProblem(
    residual,
    u,
    bcs=bcs,
    petsc_options=petsc_options,
    petsc_options_prefix="hyperelasticity",
)

In [21]:
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, : len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 4])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(
    ufl.sqrt(sum([u[i] ** 2 for i in range(len(u))])), Vs.element.interpolation_points
)
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

[2026-02-27 11:16:11.589] [info] Checking required entities per dimension
[2026-02-27 11:16:11.589] [info] Cell type: 0 dofmap: 500x27
[2026-02-27 11:16:11.589] [info] Global index computation
[2026-02-27 11:16:11.589] [info] Got 4 index_maps
[2026-02-27 11:16:11.589] [info] Get global indices


In [24]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5

v_file = VTKFile(domain.comm, "solution.pvd", "w")

for n in range(1, 21):
    B.value[2] = - rho * g * n / 20
    problem.solve()
    converged = problem.solver.getConvergedReason() > 0
    num_its = problem.solver.getIterationNumber()
    assert converged > 0, f"Solver did not converge with reason {converged}."

    print(f"Time step {n}, Number of iterations {num_its}, Load {B.value}")
    function_grid["u"][:, : len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 4])
    plotter.write_frame()
    v_file.write_function(u, float(n))
plotter.close()
v_file.close()

  0 SNES Function norm 6.983754758136e-12
Time step 1, Number of iterations 0, Load [ 0.      0.     -0.4905]
  0 SNES Function norm 1.771409300531e-01
  1 SNES Function norm 9.863546149095e+00
  2 SNES Function norm 3.432793305014e-01
  3 SNES Function norm 2.164562166966e-02
  4 SNES Function norm 7.018643530457e-06
  5 SNES Function norm 1.047150700867e-11
Time step 2, Number of iterations 5, Load [ 0.     0.    -0.981]
  0 SNES Function norm 1.771409300530e-01
  1 SNES Function norm 3.789624187513e+00
  2 SNES Function norm 6.024831320702e-02
  3 SNES Function norm 3.983121987227e-04
  4 SNES Function norm 1.846708034561e-09
Time step 3, Number of iterations 4, Load [ 0.      0.     -1.4715]
  0 SNES Function norm 1.771409300513e-01
  1 SNES Function norm 2.186229964594e+00
  2 SNES Function norm 1.866409466100e-02
  3 SNES Function norm 3.067674401899e-05
  4 SNES Function norm 1.686772320253e-11
Time step 4, Number of iterations 4, Load [ 0.     0.    -1.962]
  0 SNES Function no